In [ ]:
from skimage.io import imread
import numpy as np
from scipy.io import loadmat
from matplotlib import pyplot as plt

In [ ]:
rootfolder = ".."

Useful function for plot a 2D dictionary


In [ ]:
def get_dictionary_img(D):
    M, N = D.shape
    p = int(round(np.sqrt(M)))
    nnn = int(np.ceil(np.sqrt(N)))
    bound = 2
    img = np.ones((nnn * p + bound * (nnn - 1), nnn * p + bound * (nnn - 1)))
    for i in range(N):
        m = np.mod(i, nnn)
        n = int((i - m) / nnn)
        m = m * p + bound * m
        n = n * p + bound * n
        atom = D[:, i].reshape((p, p))
        if atom.min() < atom.max():
            atom = (atom - atom.min()) / (atom.max() - atom.min())
        img[m : m + p, n : n + p] = atom

    return img

Define a function that implements the OMP


In [ ]:
def OMP(s, D, L, tau):
#    return x

Load the image and rescale it in $[0,1]$


In [ ]:
img = imread(f"{rootfolder}/data/peppers256.png") / 255
# img = imread(f'{rootfolder}/data/barbara.png') / 255
# img = imread(f'{rootfolder}/data/Lena512.png') / 255

imsz = img.shape

# patch size
p = 8

# number of elements in the patch
M = p**2


Corrupt the image with white gaussian noise


In [ ]:
sigma_noise = 20 / 255
# noisy_img =

Percentage of removed pixel


In [ ]:
perc_of_removed_pixels = 0.25

Arbitrarily remove pixels setting them to zero


In [ ]:
# create a vector with all the indexes of the image
# idx =

# shuffle it and take the target percentage of indexes
# idx =

# the mask is 0 for the chosen idx, 1 elsewhere
# msk =

# apply the mask: set to 0 some elements in the noisy image
# noisy_img =


Compute the psnr of the noisy input


In [ ]:
# psnr_noisy =


In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(20, 10))
ax[0].imshow(noisy_img, cmap="gray")
ax[0].set_title("Noisy image before inpainting, PSNR = {psnr_noisy:.2f}")

ax[1].imshow(msk, cmap="gray")
ax[1].set_title(f"Dead pixels")


Load and display the dictionary learned from patches


In [ ]:
D = loadmat(f"{rootfolder}/data/dict_nat_img.mat")["D"]

# add a constant atom to D, KSVD was trained over patches with zero mean - and normalize it
# UPDATE D

# display the dictionary
D_img = get_dictionary_img(D)

plt.figure(figsize=(10, 10))
plt.imshow(D_img, cmap="gray")

## Inpainting


In [ ]:
# SET stopping criteria of OMP
# orthogonal matching pursuit uses sparsity and errors as stopping criteria
L = M / 2

# initialize the estimated image
img_hat = np.zeros_like(img)

# initialize the weight matrix
weights = np.zeros_like(img)

# define the step (=p for non overlapping paches)
STEP = 4  # STEP = 1 might be very time consuming, start with larger STEP

Operate patchwise


In [ ]:
for i in range(0, imsz[0] - p + 1, STEP):
    for j in range(0, imsz[1] - p + 1, STEP):
        # extrach the patch with the top left corner at pixel (ii, jj)
        # s =

        # patch extracted from the mask
        # m =

        # design the projection operator over the current patch
        # proj =

        # tau should be proportional to the number of pixels remaining in the patch
        # tau =

        # sparse coding w.r.t. PD the inpainted dictionary using L and tau as stopping criteria
        # x =

        # reconstruction: synthesis w.r.t. D the dictionary yielding sparse representation
        # s_hat =

        # use uniform weights for aggregation
        w = 1

        # put the denoised patch into the estimated image using uniform weights
        # UPDATE img_hat

        # store the weight of the current patch in the weight matrix
        # UPDATE weights


Normalize the estimated image with the computed weights


In [ ]:
# img_hat =

Compute the psnr of the estimated image


In [ ]:
# psnr_hat =

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(20, 10))

ax[0].imshow(img, cmap="gray")
ax[0].set_title(f"Original image")

ax[1].imshow(noisy_img, cmap="gray")
ax[1].set_title(f"Corrupted image, PSNR = {psnr_noisy:.2f}")

ax[2].imshow(img_hat, cmap="gray")
ax[2].set_title(f"Estimated Image,\nPSNR = {psnr_hat:.2f}")
